import requests
import pandas as pd
import matplotlib.pyplot as plt

# -----------------------------
# CONFIG
# -----------------------------
INDICATOR = "DT.DOD.DECT.CD"   # Total external debt, current USD
BASE_URL  = "https://api.worldbank.org/v2/country/all/indicator"
PER_PAGE  = 20000

# -----------------------------
# FETCH DATA
# -----------------------------
url = f"{BASE_URL}/{INDICATOR}?format=json&per_page={PER_PAGE}"

response = requests.get(url)
response.raise_for_status()
raw = response.json()[1]
df = pd.DataFrame(raw)

# -----------------------------
# CLEAN DATA
# -----------------------------
df = df[["country", "date", "value"]]
df["country"] = df["country"].apply(lambda x: x["value"])
df["date"]    = pd.to_numeric(df["date"], errors="coerce")
df["value"]   = pd.to_numeric(df["value"], errors="coerce")
df = df.dropna(subset=["value"])

# Get latest available year per country
latest = df.sort_values("date").groupby("country", as_index=False).last()

# Sort by debt descending and assign rank
latest = latest.sort_values("value", ascending=False).reset_index(drop=True)
latest["rank"] = latest.index + 1

# -----------------------------
# PLOT HORIZONTAL BAR CHART
# -----------------------------
plt.figure(figsize=(12, len(latest) * 0.15))  # dynamic height for readability

bars = plt.barh(
    latest["country"],
    latest["value"],
    color='skyblue'
)

plt.xscale("log")  # log scale is best for wide debt range
plt.xlabel("Total External Debt (USD, log scale)")
plt.ylabel("Country")
plt.title("Latest Total External Debt by Country (USD)")

# Rank inside bars (left side)
for bar, rank in zip(bars, latest["rank"]):
    plt.text(
        bar.get_width() * 0.05,  # small padding from left
        bar.get_y() + bar.get_height()/2,
        str(rank),
        va='center',
        ha='left',
        fontsize=6,
        color='black'
    )

plt.tight_layout()
plt.show()
